## Analyzing customer and product interaction

How can you boost your sales as an online retailer? The answer is simple but difficult to implement. In order to boost your sales, you have to understand your customers and their purchasing behavior. This project aims to better understand the customers of an online retailer and how we can leverage data analysis for that purpose. The dataset can be found [here](https://archive.ics.uci.edu/dataset/352/online+retail)

The goal of this project is to answer the following questions:

- Which products sell well in month x
- Why do products get canncelled and what products get cancelled.
- From which countries does the retailer get the most orders
- What are my most valuable customers?
- Which products do my most valuable customers buy?

In answering these questions with data and visualization, we can make decisions as to which products would be popular for our customer and markt them accordingly. Furthermore, we can make informed decisions about removing items which do not sell well or get canncelled often.

In [ ]:
import os

if not os.path.exists("./data/online_retail.csv"):
    print("Donwloading dataset")
    !python ./data/load_dataset.py

In [ ]:
import pandas as pd
from datetime import datetime

df_retail = pd.read_csv("./data/online_retail.csv")
regex = "%m/%d/%Y %H:%M"
df_retail["InvoiceMonth"] = df_retail["InvoiceDate"].apply(lambda x: datetime.strptime(x, regex).month)
df_retail["InvoiceYear"] = df_retail["InvoiceDate"].apply(lambda x: datetime.strptime(x, regex).year)
df_retail["Revenue"] = df_retail["UnitPrice"] * df_retail["Quantity"]
df_retail.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,InvoiceYear,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,12,2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,12,2010,20.34


In [2]:
from IPython.display import display
condition = df_retail["InvoiceNo"].astype(str).str.startswith("C")
valid_transaction = df_retail[~condition]
cancelled_transaction = df_retail[condition]
display(valid_transaction.head(2))
display(cancelled_transaction.head(2))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,InvoiceYear,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,12,2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,12,2010,20.34


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,InvoiceYear,Revenue
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom,12,2010,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom,12,2010,-4.65


In [3]:
# calculate overall revenue & quantity for each product
report_valid = valid_transaction.groupby(
    ["InvoiceYear", "InvoiceMonth", "StockCode"]
).agg({
    "Revenue": "sum",
    "Quantity": "sum"
}).reset_index()

report_cancelled = cancelled_transaction.groupby(
    ["InvoiceYear", "InvoiceMonth", "StockCode"]
).agg({
    "Revenue": "sum",
    "Quantity": "sum"
}).reset_index()

print("Report for valid transactions:")
display(report_valid.head(5))
print("Report for cancellations:")
display(report_cancelled.head(5))

Report for valid transactions:


,InvoiceYear,InvoiceMonth,StockCode,Revenue,Quantity
0,2010,12,10002,234.41,251
1,2010,12,10120,3.36,16
2,2010,12,10123C,0.65,1
3,2010,12,10124A,1.68,4
4,2010,12,10124G,2.10,5


Report for cancellations:


,InvoiceYear,InvoiceMonth,StockCode,Revenue,Quantity
0,2010,12,11001,-1.69,-1
1,2010,12,15036,-36.00,-48
2,2010,12,15056BL,-35.70,-6
3,2010,12,15056N,-35.70,-6
4,2010,12,15056P,-35.70,-6


[Try interactive version on Binder]()

In [4]:
from plots import plot_contribution
from ipywidgets import interact, Dropdown, fixed
months = sorted(df_retail["InvoiceMonth"].unique())
years = sorted(df_retail["InvoiceYear"].unique())

interact(
    plot_contribution,
    report = fixed(report_valid),
    title = fixed("Contribution Plot Valid Transactions"),
    month = Dropdown(options=months, description="Month"),
    year = Dropdown(options=years, description="Year")
)

interact(
    plot_contribution,
    report = fixed(report_cancelled),
    title = fixed("Contribution Plot Cancellations"),
    month = Dropdown(options=months, description="Month"),
    year = Dropdown(options=years, description="Year")
)

interactive(children=(Dropdown(description='Month', options=(np.int64(1), np.int64(2), np.int64(3), np.int64(4…

interactive(children=(Dropdown(description='Month', options=(np.int64(1), np.int64(2), np.int64(3), np.int64(4…

<function plots.plot_contribution(report, title: str, month: int, year: int)>